# PROBLEM STATEMENT
There is a room with a television and people come in and out of the room to watch it; the television is on only when there is at least one person in the room. 

For each person that enters the room, we record the start and end time, represented as a two-element array containing the starting time (inclusive) and ending time (exclusive), with times as integers (you can think of hours, or minutes, or any interval you like).

We want to know how long the television is on. 

## example
For instance, given the list of intervals (1 4), (6 8), (2 4), (7 9), the television is on at times 1, 2, 3 from the first interval, times 6 and 7 from the second interval, times 2 and 3 from the third interval, and times 7 and 8 from the first (last?) interval, a total of 6 times the television is on (at times 1, 2, 3, 6, 7 and 8).

*gen_intervals* function can generate random intervals

## task
Your task is to write a python function that takes a list of intervals and returns the number of times at which the television is on. 

In [1]:
from random import randint

def gen_intervals(length):
    """
    create random intervals
    """
    intervals = []
    for i in range(length):
        a = randint(i, i+5)
        b = randint(i, i+5)
        if a < b :
            intervals.append((a, b))
        elif b < a:
            intervals.append((b, a))
        else: # a == b
            intervals.append((a, b+1))
    return intervals
        
# intervals = [(1, 4), (6, 8), (2, 4), (7, 9)]
intervals = gen_intervals(10)

In [2]:
intervals

[(2, 4),
 (3, 4),
 (2, 7),
 (7, 8),
 (6, 9),
 (5, 8),
 (6, 10),
 (9, 10),
 (10, 11),
 (11, 13)]

## Not optimized function.
Create a function in the quickest way possible, not the most optimized one. The idea is to expand each interval (a, b) into the list of times [a, a+1, a+2, ..., b-1], concatenate all these expanded lists, and then count the number of unique times in the concatenated list.


### Computationl cost
n = number of intervals 

k = average or max length of intervals. In this exercise k = 5 (k << n in general)

- **expanding an interval**: O(k) since k is the worst case length of an interval (or the average length)
- **expanding all intervals**: O(n * k) since we have n intervals to expand
- **concatenating all expanded lists**: O(n * k) since we have n lists of average length k to concatenate
- **counting unique times**: O(m) where m is the length of the concatenated list, which is O(n*k) in the worst case (if all intervals are disjoint)
- **total cost**: O(n * k) + O(n * k) + O(n * k) = O(n * k). Since k is fixed and small compared to n, we can simplify this to O(n).


Moreover, this function allocate really "huge" lists in memory, which is not optimal.





In [3]:
from typing import List, Tuple
from functools import wraps
from time import time

In [4]:
def time_tracker(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        
        start = time()
        result = func(*args, **kwargs)
        end = time()
        print(f"Function {func.__name__} took {end - start:.6f} seconds")
        return result, end-start
    return wrapper

In [5]:
@time_tracker
def count_times(intervals: List[Tuple[int, int]]) -> int:
    """
    Count the total length covered by the given intervals. This will count the total time the television is on.

    Args:
        intervals (List[Tuple[int, int]]): list of tuples (a,b) representing intervals [a,b)

    Returns:
        int: total time covered by the intervals
    """
    
    def expand_interval(interval: List[int]) -> List[int]:
        """
        Expand an interval [a,b) into a list of integers [a, a+1, ..., b-1]

        Args:
            interval (List[int]): interval [a,b)

        Returns:
            List[int]: list of integers [a, a+1, ..., b-1]
        """
        return list(range(interval[0], interval[1]))
    
    if not intervals:
        return 0
        
    return len(set(sum(map(expand_interval, intervals), [])))


count_times(intervals)[0]

Function count_times took 0.000011 seconds


11

## Optimized function
Let's suppose *n* and *k* very big ( k >> logn). Let's try to optimize the function above. The idea is exploiting the sorting (computational cost O(nlogn) and it is very optimized since it exploits C which is fast) and finding the overlapping intervals. Then sum up the lengths of the new non-overlapping intervals.


### Computationl cost
n = number of intervals 

k = average or max length of intervals. In this exercise k = 5(k << n in general)

- **sorting**: O(nlogn)
- **'for' cycle**: O(n) since we have n intervals to check
- **total cost**: O(nlogn) + O(n) = O(nlogn) since nlogn is the dominant term.




In [6]:
@time_tracker
def optimized_count_times(intervals: List[Tuple[int, int]]) -> int:
    """
    Optimized version: count the total length covered by the given intervals. This will count the total time the television is on.
    1) Sort the intervals by their start time.
    2) Merge overlapping intervals.
    3) Sum the lengths of the merged intervals.

    Args:
        intervals (List[Tuple[int, int]]): list of tuples (a,b) representing intervals [a,b)

    Returns:
        int: total time covered by the intervals
    """
    if not intervals:
        return 0
    

    intervals.sort(key=lambda x: x[0]) # Thus each overlap can only be with the next interval

    merged = []
    prev_start, prev_end = intervals[0]

    for start, end in intervals[1:]:
        if start <= prev_end:  # overlap
            prev_end = max(prev_end, end)
        else:  # no overlap, save previous
            merged.append((prev_start, prev_end))
            prev_start, prev_end = start, end


    merged.append((prev_start, prev_end))


    return sum(end - start for start, end in merged)
            
    
    
optimized_count_times(intervals)[0]


Function optimized_count_times took 0.000009 seconds


11

In [7]:

for n in range (5, 10001, 100):
    print("Number of intervals:", n)
    intervals = gen_intervals(n)


    op_count = optimized_count_times(intervals)
    count = count_times(intervals)

    assert op_count[0] == count[0], f"Optimized: {op_count[0]}, Original: {count[0]}"
    
    print("-"*40)
    print("\n")




Number of intervals: 5
Function optimized_count_times took 0.000006 seconds
Function count_times took 0.000007 seconds
----------------------------------------


Number of intervals: 105
Function optimized_count_times took 0.000021 seconds
Function count_times took 0.000047 seconds
----------------------------------------


Number of intervals: 205
Function optimized_count_times took 0.000034 seconds
Function count_times took 0.000097 seconds
----------------------------------------


Number of intervals: 305
Function optimized_count_times took 0.000227 seconds
Function count_times took 0.000458 seconds
----------------------------------------


Number of intervals: 405
Function optimized_count_times took 0.000108 seconds
Function count_times took 0.000295 seconds
----------------------------------------


Number of intervals: 505
Function optimized_count_times took 0.000217 seconds
Function count_times took 0.002127 seconds
----------------------------------------


Number of interval